In [ ]:
! pip install torchinfo

In [ ]:
import polars as pl
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification
import json
import re
import numpy as np
import torch
import gc
import itertools
import os
import torch.nn as nn
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Loading the necessary datasets
# Dimitri's
noteevents_tok = pl.read_csv('/content/drive/MyDrive/CS544 Final Project Data/NOTEEVENTS_TOK_PAD.csv', has_header=True)
diagnosis_vocab = json.load(open('/content/drive/MyDrive/CS544 Final Project Data/diagnosis_vocab_PAD.json', 'r'))
procedure_vocab = json.load(open('/content/drive/MyDrive/CS544 Final Project Data/procedure_vocab_PAD.json', 'r'))

# Izabel's
# noteevents_tok = pl.read_csv('/content/drive/MyDrive/cs 544/Final project/CS544 Final Project Data/NOTEEVENTS_TOK_PAD.csv', has_header=True)
# diagnosis_vocab = json.load(open('/content/drive/MyDrive/cs 544/Final project/CS544 Final Project Data/diagnosis_vocab_PAD.json', 'r'))
# procedure_vocab = json.load(open('/content/drive/MyDrive/cs 544/Final project/CS544 Final Project Data/procedure_vocab_PAD.json', 'r'))
# noteevents_tok.head()

In [ ]:
noteevents_tok = noteevents_tok.with_columns([
    pl.col("TEXT").map_elements(lambda x: eval(x), return_dtype=pl.List(pl.Int64)).alias("TEXT"),
    pl.col("ICD9_DIAG").map_elements(lambda x: eval(x), return_dtype=pl.List(pl.Int64)).alias("ICD9_DIAG"),
    pl.col("ICD9_PROC").map_elements(lambda x: eval(x), return_dtype=pl.List(pl.Int64)).alias("ICD9_PROC")
])
noteevents_tok.head()

SUBJECT_ID,TEXT,ICD9_PROC,ICD9_DIAG
i64,list[i64],list[i64],list[i64]
24515,"[101, 24930, … 102]","[1, 2, … 0]","[1, 2, … 0]"
56506,"[101, 24930, … 102]","[7, 0, … 0]","[6, 7, … 0]"
76558,"[101, 24930, … 102]","[2, 8, … 0]","[6, 29, … 0]"
20341,"[101, 24930, … 102]","[1, 2, … 0]","[53, 54, … 0]"
7744,"[101, 24930, … 102]","[14, 0, … 0]","[2, 58, … 0]"


In [ ]:
icd9_diag = noteevents_tok['ICD9_DIAG']
icd9_proc = noteevents_tok['ICD9_PROC']
diag_pad_length = icd9_diag.list.len().max()
proc_pad_length = icd9_proc.list.len().max()
(diag_pad_length, proc_pad_length)

(144, 50)

In [ ]:
icd9_diag = noteevents_tok['ICD9_DIAG']
icd9_proc = noteevents_tok['ICD9_PROC']
min_diag_pad_length = icd9_diag.list.len().min()
min_proc_pad_length = icd9_proc.list.len().min()
(min_diag_pad_length, min_proc_pad_length)

(144, 50)

In [ ]:
# DONT RUN THIS WILL WRITE THE FILES AGAIN
noteevents_tok = noteevents_tok.with_columns([
    pl.col('TEXT').map_elements(lambda x: f"{x.to_list()}", return_dtype=pl.Utf8).alias('TEXT'),
    pl.col('ICD9_DIAG').map_elements(lambda x: f"{x.to_list()}", return_dtype=pl.Utf8).alias('ICD9_DIAG'),
    pl.col('ICD9_PROC').map_elements(lambda x: f"{x.to_list()}", return_dtype=pl.Utf8).alias('ICD9_PROC')
])
noteevents_tok.write_csv('/content/drive/MyDrive/CS544 Final Project Data/NOTEEVENTS_TOK_PAD.csv')
json.dump(diagnosis_vocab, open('/content/drive/MyDrive/CS544 Final Project Data/diagnosis_vocab_PAD.json', 'w'))
json.dump(procedure_vocab, open('/content/drive/MyDrive/CS544 Final Project Data/procedure_vocab_PAD.json', 'w'))

In [ ]:
embedding_model = AutoModelForTokenClassification.from_pretrained("samrawal/medical-sentence-tokenizer").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [ ]:
embedding_model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [ ]:
example_tensor =torch.tensor(noteevents_tok["TEXT"].to_list()[0], dtype=torch.long).reshape(1, 512)
example_tensor

tensor([[  101, 24930, 11234, 14265,   131,   164,   115,   115, 21640,  1571,
           118,   124,   118,  1695,   115,   115,   166, 14856,  7111,  2176,
         14265,   131,   164,   115,   115, 21640,  1571,   118,   124,   118,
          1853,   115,   115,   166, 14265,  1104, 20729,   131,   164,   115,
           115, 19980,  1475,   118,   124,   118,   123,   115,   115,   166,
          9850,   131,   150,  2516,   131, 10103, 26423, 16426,  6617,  1665,
         24890, 17444,  2271, 18732, 12347, 10783, 11607,  1942,   131, 20394,
          2556,  2489,   119,   145,  6258, 18082,  2069,  3663, 11345, 11629,
          9919, 11680,  1942, 15393,  2162, 22680, 12480,   131,  1109,  5351,
          1110,   170,  4335,   118,  1214,   118,  1385,  2581,  1150,  2756,
          1106,  1126,  1796,  2704,  1113,   164,   115,   115,   124,   118,
          1626,   115,   115,   166, 19533,  1104,   174,  8508, 11305, 11048,
          2489, 26120,  1205,  1117,  1286,  1981,  

In [ ]:
example_tensor.shape

torch.Size([1, 512])

In [ ]:
example_embedding = embedding_model(example_tensor)

In [ ]:
example_embedding

TokenClassifierOutput(loss=None, logits=tensor([[[-4.5740,  5.1140, -0.7754],
         [ 8.7543, -3.8294, -4.0519],
         [ 7.2435, -2.6415, -3.7899],
         ...,
         [-4.8048, -3.3816,  9.0521],
         [-4.5775, -3.7425,  9.1542],
         [-5.0736, -2.4700,  8.5366]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class NoteEventsDataset(Dataset):
    def __init__(self, data):
        self.data = {
            "TEXT": data["TEXT"].to_list(),
            "ICD9_DIAG": data["ICD9_DIAG"].to_list(),
            "ICD9_PROC": data["ICD9_PROC"].to_list()
        }

    def __len__(self):
        return len(self.data["TEXT"])

    def __getitem__(self, idx):
        text = torch.tensor(self.data["TEXT"][idx], dtype=torch.float32)
        icd9_diag = torch.tensor(self.data["ICD9_DIAG"][idx], dtype=torch.long)
        icd9_proc = torch.tensor(self.data["ICD9_PROC"][idx], dtype=torch.long)
        return {
            "TEXT": text,
            "ICD9_DIAG": icd9_diag,
            "ICD9_PROC": icd9_proc
        }

In [ ]:
train, test = train_test_split(noteevents_tok, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
trainloader = DataLoader(NoteEventsDataset(train), batch_size=32, shuffle=True)
valloader = DataLoader(NoteEventsDataset(val), batch_size=32, shuffle=False)
testloader = DataLoader(NoteEventsDataset(test), batch_size=32, shuffle=False)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for batch in trainloader:
    print(batch["TEXT"].shape)
    break

torch.Size([32, 512])


In [ ]:
class TextToDualVectorsModel(nn.Module):
    def __init__(
        self,
        diagnosis_vocab_size=30000,   # First output vocabulary size
        procedure_vocab_size=10000,   # Second output vocabulary size
        input_dim=512,              # Dimension of pre-embedded input tokens
        hidden_dim=256,             # Dimension of hidden layers
        dropout_rate=0.1,           # Dropout probability
        pad_token_id=0,             # ID of padding token
    ):
        super(TextToDualVectorsModel, self).__init__()
        self.pad_token_id = pad_token_id

        # Linear projection for pre-embedded input tokens
        self.input_projection = nn.Linear(input_dim, hidden_dim)

        # Bidirectional LSTM for encoding input sequence
        self.encoder = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=dropout_rate
        )

        # Feature extraction using attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,  # * 2 for bidirectional
            num_heads=8,
            dropout=dropout_rate
        )

        # Output heads for the two vector sequences
        self.output_head1 = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
        )

        self.output_head2 = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
        )

        # Final projection layers to vocabularies
        self.project_to_vocab1 = nn.Linear(hidden_dim // 2, diagnosis_vocab_size)
        self.project_to_vocab2 = nn.Linear(hidden_dim // 2, procedure_vocab_size)

        # Output sequence generators
        self.generator1 = nn.LSTM(
            input_size=hidden_dim // 2,
            hidden_size=hidden_dim // 2,
            num_layers=2,
            batch_first=True,
            dropout=dropout_rate
        )

        self.generator2 = nn.LSTM(
            input_size=hidden_dim // 2,
            hidden_size=hidden_dim // 2,
            num_layers=2,
            batch_first=True,
            dropout=dropout_rate
        )

    def create_padding_mask(self, x):
        """
        Create a mask for padding positions.
        Assumes input is tokenized and pad tokens have value pad_token_id.

        Args:
            x: Input tensor of shape (batch_size, seq_len)

        Returns:
            Padding mask of shape (batch_size, seq_len, 1)
        """
        # For tokenized input: mask positions with pad_token_id
        if x.dtype == torch.long:
            return (x == self.pad_token_id).float().unsqueeze(-1)

        # For pre-embedded inputs: assume zeros represent padding
        # This is a heuristic - adjust based on your data specifics
        return (torch.sum(torch.abs(x), dim=-1) == 0).float().unsqueeze(-1)

    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x: Input tensor of shape (batch_size, 1, 512) or (batch_size, 1, 512, input_dim)
               If x contains token IDs, they're treated as indices only for padding mask
               If x contains embeddings, they're processed directly

        Returns:
            output1: First output sequence of shape (batch_size, 144) with indices from vocab1
            output2: Second output sequence of shape (batch_size, 50) with indices from vocab2
        """
        # Implementation similar to forward, but returning logits as well
        x = x.squeeze(1)  # Remove the sequence dimension
        batch_size = x.shape[0]

        # Determine input type and create appropriate padding mask
        if len(x.shape) == 2:  # Tokenized input
            padding_mask = self.create_padding_mask(x)
            if x.dtype == torch.long:
                x = x.float()
            if len(x.shape) == 2:
                x = x.unsqueeze(-1)
                x = x.repeat(1, 1, self.input_projection.in_features)
        else:  # Pre-embedded input
            padding_mask = self.create_padding_mask(x)

        projected_input = self.input_projection(x)
        encoder_outputs, _ = self.encoder(projected_input)

        encoder_outputs_t = encoder_outputs.transpose(0, 1)

        # Ensure key_padding_mask has shape (batch_size, seq_len) or (batch_size)
        # by adding an extra dimension if necessary

        # key_padding_mask = (padding_mask == 1).squeeze(-1)
        key_padding_mask = (padding_mask.squeeze(-1) == 1) #  key_padding_mask = (padding_mask == 1).squeeze(-1) shape: (batch_size, seq_len)
        if key_padding_mask.dim() == 1:  # If key_padding_mask is 1D after squeeze
            key_padding_mask = key_padding_mask.unsqueeze(1).repeat(1, x.shape[1])  # Reshape to (batch_size, seq_len)


        # print("x.shape", x.shape)
        # print("padding_mask.shape", padding_mask.shape)
        # print("key_padding_mask.shape", key_padding_mask.shape)


        # if len(key_padding_mask.shape) == 1:
        #     key_padding_mask = key_padding_mask[:, None]  # Add extra dimension

        attended_outputs, _ = self.attention(
            encoder_outputs_t,
            encoder_outputs_t,
            encoder_outputs_t,
            key_padding_mask=key_padding_mask
        )
        attended_outputs = attended_outputs.transpose(0, 1)

        # Global representation of the input (average pooling)
        # Ensure padding_mask has the correct number of dimensions

        # if len(padding_mask.shape) == 2:  # If padding_mask has shape (batch_size, seq_len)
        #     padding_mask = padding_mask.unsqueeze(-1)  # Add extra dimension
        # Correct the dimension of padding_mask
        # The expected shape is (batch_size, seq_len, 1) or (batch_size, 1, seq_len)
        # padding_mask = padding_mask.unsqueeze(2)  # Correct to (batch_size, seq_len, 1)

        # Correct padding_mask shape
        if padding_mask.dim() == 2:
            padding_mask = padding_mask.unsqueeze(2)
        else:
            padding_mask = padding_mask.transpose(1, 2)


        global_repr = torch.mean(attended_outputs * (1 - padding_mask), dim=1)

        head1_features = self.output_head1(global_repr)
        head2_features = self.output_head2(global_repr)

        seq1_input = head1_features.unsqueeze(1).repeat(1, 144, 1)
        seq1_outputs, _ = self.generator1(seq1_input)
        logits1 = self.project_to_vocab1(seq1_outputs)

        seq2_input = head2_features.unsqueeze(1).repeat(1, 50, 1)
        seq2_outputs, _ = self.generator2(seq2_input)
        logits2 = self.project_to_vocab2(seq2_outputs)

        output1 = torch.argmax(logits1, dim=-1)
        output2 = torch.argmax(logits2, dim=-1)

        return (output1, output2), (logits1, logits2)

    def forward_with_logits(self, x):
        """
        Forward pass that returns both token indices and logits.
        Useful for training.
        """
        # Implementation similar to forward, but returning logits as well
        x = x.squeeze(1)  # Remove the sequence dimension
        batch_size = x.shape[0]

        # Determine input type and create appropriate padding mask
        if len(x.shape) == 2:  # Tokenized input
            padding_mask = self.create_padding_mask(x)
            if x.dtype == torch.long:
                x = x.float()
            if len(x.shape) == 2:
                x = x.unsqueeze(-1)
                x = x.repeat(1, 1, self.input_projection.in_features)
        else:  # Pre-embedded input
            padding_mask = self.create_padding_mask(x)

        projected_input = self.input_projection(x)
        encoder_outputs, _ = self.encoder(projected_input)

        encoder_outputs_t = encoder_outputs.transpose(0, 1)

        # Ensure key_padding_mask has shape (batch_size, seq_len) or (batch_size)
        # by adding an extra dimension if necessary

        # key_padding_mask = (padding_mask == 1).squeeze(-1)
        key_padding_mask = (padding_mask.squeeze(-1) == 1) #  key_padding_mask = (padding_mask == 1).squeeze(-1) shape: (batch_size, seq_len)
        if key_padding_mask.dim() == 1:  # If key_padding_mask is 1D after squeeze
            key_padding_mask = key_padding_mask.unsqueeze(1).repeat(1, x.shape[1])  # Reshape to (batch_size, seq_len)


        # print("x.shape", x.shape)
        # print("padding_mask.shape", padding_mask.shape)
        # print("key_padding_mask.shape", key_padding_mask.shape)


        # if len(key_padding_mask.shape) == 1:
        #     key_padding_mask = key_padding_mask[:, None]  # Add extra dimension

        attended_outputs, _ = self.attention(
            encoder_outputs_t,
            encoder_outputs_t,
            encoder_outputs_t,
            key_padding_mask=key_padding_mask
        )
        attended_outputs = attended_outputs.transpose(0, 1)

        # Global representation of the input (average pooling)
        # Ensure padding_mask has the correct number of dimensions

        # if len(padding_mask.shape) == 2:  # If padding_mask has shape (batch_size, seq_len)
        #     padding_mask = padding_mask.unsqueeze(-1)  # Add extra dimension
        # Correct the dimension of padding_mask
        # The expected shape is (batch_size, seq_len, 1) or (batch_size, 1, seq_len)
        # padding_mask = padding_mask.unsqueeze(2)  # Correct to (batch_size, seq_len, 1)

        # Correct padding_mask shape
        if padding_mask.dim() == 2:
            padding_mask = padding_mask.unsqueeze(2)
        else:
            padding_mask = padding_mask.transpose(1, 2)


        global_repr = torch.mean(attended_outputs * (1 - padding_mask), dim=1)

        head1_features = self.output_head1(global_repr)
        head2_features = self.output_head2(global_repr)

        seq1_input = head1_features.unsqueeze(1).repeat(1, 144, 1)
        seq1_outputs, _ = self.generator1(seq1_input)
        logits1 = self.project_to_vocab1(seq1_outputs)

        seq2_input = head2_features.unsqueeze(1).repeat(1, 50, 1)
        seq2_outputs, _ = self.generator2(seq2_input)
        logits2 = self.project_to_vocab2(seq2_outputs)

        output1 = torch.argmax(logits1, dim=-1)
        output2 = torch.argmax(logits2, dim=-1)

        return (output1, output2), (logits1, logits2)

    def train_and_validate(self, model, train_dataloader, val_dataloader, epochs=10):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        optimizer = torch.optim.Adam(model.parameters(), lr=4e-3)
        criterion = DualSequenceLoss()
        model.to(device)

        for epoch in tqdm(range(epochs)):
            # Training
            model.train()
            train_loss = 0
            for batch in tqdm(train_dataloader):
                inputs = batch["TEXT"].to(device)  # This could be tokens or embeddings
                ICD9_DIAG = batch["ICD9_DIAG"].to(device)   # Target from vocab1
                ICD9_PROC = batch["ICD9_PROC"].to(device)   # Target from vocab2

                optimizer.zero_grad()

                # Get both indices and logits for efficient training
                (output1, output2), (logits1, logits2) = model.forward_with_logits(inputs)

                # Calculate loss
                loss = criterion(logits1, ICD9_DIAG, logits2, ICD9_PROC)

                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            # Validation
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch in val_dataloader:
                    inputs = batch["TEXT"].to(device)
                    ICD9_DIAG = batch["ICD9_DIAG"].to(device)
                    ICD9_PROC = batch["ICD9_PROC"].to(device)

                    # Get both indices and logits
                    (output1, output2), (logits1, logits2) = model.forward_with_logits(inputs)

                    loss = criterion(logits1, ICD9_DIAG, logits2, ICD9_PROC)
                    val_loss += loss.item()

            print(f"Epoch {epoch+1}/{epochs}")
            print(f"Training loss: {train_loss/len(train_dataloader)}")
            print(f"Validation loss: {val_loss/len(val_dataloader)}")

        return model

In [ ]:
class DualSequenceLoss(nn.Module):
    def __init__(self, pad_token_id=0):
        super(DualSequenceLoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, logits1, ICD9_DIAG, logits2, ICD9_PROC):
        # logits1: (batch_size, 144, diagnosis_vocab_size)
        # ICD9_DIAG: (batch_size, 144)
        # logits2: (batch_size, 50, procedure_vocab_size)
        # ICD9_PROC: (batch_size, 50)

        # Reshape for CrossEntropyLoss
        batch_size = logits1.size(0)

        ICD9_DIAG = torch.clamp(ICD9_DIAG, 0, logits1.shape[-1] - 1)
        ICD9_PROC = torch.clamp(ICD9_PROC, 0, logits2.shape[-1] - 1)

        loss1 = self.criterion(
            logits1.view(batch_size * 144, -1),
            ICD9_DIAG.view(-1)
        )
        loss2 = self.criterion(
            logits2.view(batch_size * 50, -1),
            ICD9_PROC.view(-1)
        )
        # You can adjust the weighting between the two losses
        return (loss1 + loss2)/2

In [ ]:
# Initialize model with only output vocabularies
model = TextToDualVectorsModel(
    diagnosis_vocab_size=len(diagnosis_vocab),  # First output vocab size
    procedure_vocab_size=len(procedure_vocab),  # Second output vocab size
    input_dim=512           # Assuming input is embedding dimension or token feature dimension
).to(device)

In [ ]:
from torchinfo import summary
summary(model, input_size=(32, 1, 512)) # 32 for default batch size

Layer (type:depth-idx)                   Output Shape              Param #
TextToDualVectorsModel                   [32, 144]                 --
├─Linear: 1-1                            [32, 512, 256]            131,328
├─LSTM: 1-2                              [32, 512, 512]            2,629,632
├─MultiheadAttention: 1-3                [512, 32, 512]            1,050,624
├─Sequential: 1-4                        [32, 128]                 --
│    └─Linear: 2-1                       [32, 256]                 131,328
│    └─LayerNorm: 2-2                    [32, 256]                 512
│    └─ReLU: 2-3                         [32, 256]                 --
│    └─Dropout: 2-4                      [32, 256]                 --
│    └─Linear: 2-5                       [32, 128]                 32,896
│    └─LayerNorm: 2-6                    [32, 128]                 256
│    └─ReLU: 2-7                         [32, 128]                 --
│    └─Dropout: 2-8                      [32, 128]     

In [ ]:
model.train_and_validate(model, trainloader, valloader, epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2644 [00:00<?, ?it/s]

Epoch 1/10
Training loss: 0.7005659766303609
Validation loss: 0.6700891191858387


  0%|          | 0/2644 [00:00<?, ?it/s]

Epoch 2/10
Training loss: 0.6717934054794542
Validation loss: 0.6680480262966272


...

  0%|          | 0/2644 [00:00<?, ?it/s]

Epoch 9/10
Training loss: 0.6693111660125216
Validation loss: 0.6685204166417403


  0%|          | 0/2644 [00:00<?, ?it/s]

Epoch 10/10
Training loss: 0.6690963440082979
Validation loss: 0.66882328673318


TextToDualVectorsModel(
  (input_projection): Linear(in_features=512, out_features=256, bias=True)
  (encoder): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (output_head1): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
  )
  (output_head2): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): 

In [ ]:
## Izabel's
# Save model
#torch.save(model.state_dict(), '/content/drive/MyDrive/cs 544/Final project/final_project_model.pth')
# Load model
#model.load_state_dict(torch.load('/content/drive/MyDrive/cs 544/Final project/final_project_model.pth'))

##Dimitri's
# Save model
torch.save(model.state_dict(), '/content/drive/MyDrive/Final project/final_project_model2.pth')
# Load model
model.load_state_dict(torch.load('/content/drive/MyDrive/Final project/final_project_model2.pth'))


<All keys matched successfully>

In [ ]:
# Testing metrics accuracy, recall, precision and f1 score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_predict = []
all_labels = []

with torch.no_grad():
  for batch in testloader:
    inputs = batch["TEXT"].to(device)
    icd9_diag = batch["ICD9_DIAG"].to(device)
    icd9_proc = batch["ICD9_PROC"].to(device)
    label = torch.cat((icd9_diag, icd9_proc), dim=1)


    outputs, logits = model(inputs)
    output1, output2 = outputs
    logits1, logits2 = logits

    logits1 = logits1.argmax(dim=-1)
    logits2 = logits2.argmax(dim=-1)

    predicted = torch.cat((logits1, logits2), dim=1)

    all_predict.extend(predicted.cpu().numpy())
    all_labels.extend(label.cpu().numpy())

In [ ]:
all_predict[0]

array([   6,   60, 2030, 2030,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,    0,    0,    0,    0,    0,   

In [ ]:
all_labels[0]

array([ 862, 1022,  176,   47, 3943,   17,    2, 1598,  130,  115,  189,
        841,  139,    5,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,  182,  150,   10,    3,    6,   

In [ ]:
# Getting metrics
from sklearn.metrics import multilabel_confusion_matrix, classification_report

# Assuming 'all_labels' and 'all_predict' are lists of lists
# First, flatten the lists if they are not already
all_labels_flat = [item for sublist in all_labels for item in sublist]
all_predict_flat = [item for sublist in all_predict for item in sublist]

# Calculate the confusion matrix for each label
# cm = multilabel_confusion_matrix(all_labels, all_predict) # use this for list of lists

cm = multilabel_confusion_matrix(all_labels_flat, all_predict_flat)  # Assuming flattened list

accuracy_list = []
recall_list = []
precision_list = []
f1_score_result_list = []

# Iterate through each label and calculate/print metrics
for i in range(cm.shape[0]):  # cm.shape[0] gives the number of labels
    tn, fp, fn, tp = cm[i].ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0  # Handle zero division
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0  # Handle zero division
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0  # Handle zero division
    accuracy_list.append(accuracy)
    if recall != 0:
      recall_list.append(recall)
    if precision != 0:
      precision_list.append(precision)  # Handle zero division
    if f1_score != 0:
      f1_score_result_list.append(f1_score)

print("Metrics:")
print("Accuracy:", np.average(accuracy_list))
print("Recall:", np.average(recall_list))
print("Precision:", np.average(precision_list))
print("F1 Score:", np.average(f1_score_result_list))

# # You can also generate a classification report for all labels
# report = classification_report(all_labels_flat, all_predict_flat, zero_division=0)
# print("\nClassification Report for all labels (Flattened list):")
# print(report)

Metrics:
Accuracy: 0.9999663365425376
Recall: 0.4997148084063202
Precision: 0.2503638054006883
F1 Score: 0.29891474895800185


In [ ]:
num_correct = 0
misclassifications = 0

for pred, actual in zip(all_predict, all_labels):

  diag_pred = pred[:144]
  diag_actual = actual[:144]

  proc_pred = pred[144:]
  proc_actual = actual[144:]

  diag_pred = set(diag_pred) # get unique token ids
  diag_actual = set(diag_actual)
  diag_actual.discard(0) # discard padding tokens
  diag_pred.discard(0) # discard padding tokens

  proc_pred = set(proc_pred) # get unique token ids
  proc_actual = set(proc_actual)
  proc_actual.discard(0) # discard padding tokens
  proc_pred.discard(0) # discard padding tokens

  for i in diag_pred:
    if i in diag_actual:
      num_correct += 1
    else:
      misclassifications += 1
  for i in proc_pred:
    if i in proc_actual:
      num_correct += 1
    else:
      misclassifications += 1


total_classifications = num_correct + misclassifications
num_correct /= total_classifications
misclassifications /= total_classifications
print("Accuracy:", num_correct)
print("Misclassifications:", misclassifications)

Accuracy: 0.12067334972574238
Misclassifications: 0.8793266502742576


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Calculate metrics
accuracy = accuracy_score(all_labels_flat, all_predict_flat)
recall = recall_score(all_labels_flat, all_predict_flat, average='weighted', zero_division=0)
precision = precision_score(all_labels_flat, all_predict_flat, average='weighted', zero_division=0)
f1 = f1_score(all_labels_flat, all_predict_flat, average='weighted', zero_division=0)

print("Metrics:")
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)


Metrics:
Accuracy: 0.9022749829868634
Recall: 0.9022749829868634
Precision: 0.8335852931014872
F1 Score: 0.866207156285333
